In [123]:
import json
import numpy as np
from statistics import mean
import random

In [127]:
class ACO:
    #ACO Parameters
    alpha = 1
    beta = 5
    rho = 0.4
    rho1 = 0.5
    m = 10
    q0 = 0.9
    Nmax = 30
    w1 = 0.5
    w2 = 0.5
    
    #Data
    Tasks = []
    task_len = 0
    VMs = []
    vm_len = 0
    
    #Matrix to Store the task-vm pheromone
    Pher = 0
    
    #Execution Times of the Tasks
    Et = 0
    
    #Transmission times of the tasks
    Tt = 0
    
    # Total runtime  = Transmission + Execution Times
    TRT = 0
    
    # VM Computation 
    Vm_comp = 0
    vm_comp_avg = 0
    
    #Sum of all the execution times
    total_exec_time = 0
    #Sum of all input file sizes
    total_file_size = 0
    # Max Computation power VM , Min Computation Power VM
    max_comp = 0
    min_comp = 0
    # Bandwidth of the one with max and min comp power
    max_comp_bw = 0
    min_comp_bw = 0
    # C_Time_min and C_Time_max
    c_time_min = 0
    c_time_max = 0
    # Max cost VM cost, min cost vm cost
    min_ucost = 0
    max_ucost = 0
    # TotalCostmin and max
    total_cost_min = 0
    total_cost_max = 0
    
    # Related to the Current Iteration
    M = 0 #Mapping from tasks to VMs: [n_ants, n_tasks, n_vms]
    vm_completetime = 0 # [n_ants, n_vms] Sum of execution times for all tasks assigned to it
    c_time_i = 0 # Completion times for the set of tasks [n_ants]
    total_cost_i = 0 # Cost of using VMs [n_ants]  
    time_cf = 0 # Time Constraint Function: [n_ants]
    c_cost_i = 0 # Cost Constraint Function: [n_ants]
    L = 0 # w1*time_cf + w2*c_cost_i: [n_ants]
    
    
    L_global_opt = -1
    M_global_opt = -1 # [n_tasks, n_vms]
    current_total_exectime = 0 # [n_ants] total execution time till now over all VMs
    
    #Mostly wont need these as variables:
    # Completion time of a VM (Sum of runtimes of all tasks assigned to it)
    # VM_completetime = []
    # Total Completion time
    # Max_VM_completetime
    # Cost of completing all tasks
    # Sum over all VM ( Unit cost of VM * Max time to complete )
    # Cost Constraint Function, Time constraint function, overall objective function
    
    #Laod the data
    def loadData(self, vm_file, task_file):
        fvm = open(vm_file, "r")
        ftask = open(task_file, "r")
        vm_data = (fvm.read())
        task_data = (ftask.read())
        self.VMs = json.loads(vm_data)
        self.Tasks = json.loads(task_data)
        
        self.calcValues()
        
        
    def calcValues(self):
        self.task_len = len(self.Tasks)
        self.vm_len = len(self.VMs)
        
        self.setVmCompCost()
        
        self.setTotalTimes()
        self.setCTimes()
        
        self.setInitialPheromone()
    
    
    def setTotalTimes(self):
        self.setExecutionTimes()
        self.setTransmissionTimes()
        
        self.TRT = self.Et + self.Tt
    
    def setVmCompCost(self):
        self.Vm_comp = np.zeros((self.vm_len))
        self.Vm_comp[0] = self.VMs[0]['mips'] * self.VMs[0]['n_pe']
        
        self.max_comp = self.Vm_comp[0]
        self.max_comp_bw = self.VMs[0]["bw"]
        self.min_comp = self.Vm_comp[0]
        self.min_comp_bw = self.VMs[0]["bw"]
        
        self.max_ucost = self.VMs[0]['cost']
        self.min_ucost = self.VMs[0]['cost']
        self.vm_comp_avg = self.Vm_comp[0]
        
        for i in range(1, self.vm_len):
            self.Vm_comp[i] = self.VMs[i]['mips'] * self.VMs[i]['n_pe']
            self.vm_comp_avg += self.Vm_comp[i]
            if (self.Vm_comp[i] > self.max_comp):
                self.max_comp = self.Vm_comp[i]
                self.max_comp_bw = self.VMs[i]['bw']
            if (self.Vm_comp[i] < self.min_comp):
                self.min_comp = self.Vm_comp[i]
                self.min_comp_bw = self.VMs[i]['bw']
                
            if (self.VMs[i]['cost'] < self.min_ucost):
                self.min_ucost  = self.VMs[i]['cost']
            if (self.VMs[i]['cost'] > self.max_ucost):
                self.max_ucost  = self.VMs[i]['cost']
                
        self.vm_comp_avg = self.vm_comp_avg / float(self.vm_len)
        
    def setExecutionTimes(self):
        #Uses length of the task and processing power of the VM
        # L/P
        # P = mips*n_pe
        
        self.Et = np.zeros((self.task_len, self.vm_len))
        for i in range(self.task_len):
            for j in range(self.vm_len):
                p = self.Vm_comp[j]
                self.Et[i][j] = self.Tasks[i]["length"] / float(p)
            self.total_exec_time += self.Tasks[i]["length"]
    
    def setTransmissionTimes(self):
        # Uses file size of task and bandwidth of vm
        # fs/bw
        self.Tt = np.zeros((self.task_len, self.vm_len))
        for i in range(self.task_len):
            for j in range(self.vm_len):
                self.Tt[i][j] = self.Tasks[i]["file"] / float(self.VMs[j]["bw"])
            self.total_file_size += self.Tasks[i]["file"]
            
    def setCTimes(self):
        self.c_time_min = ((self.total_exec_time) / (self.vm_len * self.max_comp)) + (self.total_file_size/ (self.vm_len * self.max_comp_bw)) 
        self.c_time_max = ((self.total_exec_time) / (self.vm_len * self.min_comp)) + (self.total_file_size/ (self.vm_len * self.min_comp_bw))
    
        self.total_cost_min = self.c_time_min * self.min_ucost
        self.total_cost_max = self.c_time_max * self.max_ucost
        
    def setInitialPheromone(self):
        # vm comp / average comp
        self.Pher = np.zeros((self.task_len,self.vm_len))
        for j in range(self.vm_len):
            t0 = self.Vm_comp[j] / float(self.vm_comp_avg)
            for i in range(self.task_len):
                self.Pher[i][j] = t0
    
    def chooseVM(self, tid, a):
        if (tid == 0):
            return random.randint(0, self.vm_len-1)
        
        q = random.random()
        selectedvm = 0
        load = np.zeros((self.vm_len))
        heuristic = np.zeros((self.vm_len))
        total_heuristic = 0.0
        
        avg_e = self.current_total_exectime[a] / float(self.vm_len)
        for l in range(self.vm_len):
            load[l] = 1.0 - ((self.vm_completetime[a][l] - avg_e)/self.current_total_exectime[a])
            heuristic[l] = pow(self.Pher[tid][l], self.alpha) * pow(load[l]/self.Et[tid][l], self.beta)
            total_heuristic += heuristic[l]
            
        if (q < self.q0):
            selectedvm = 0
            maxv = heuristic[0]
            for v in range(self.vm_len):
                if (heuristic[v] > maxv):
                    selectedvm = v
                    maxv = heuristic[v]
            return selectedvm
        else:
            limits = np.zeros(self.vm_len)
            
            cumul = 0.0
            for i in range(self.vm_len):
                limits[i] = 1000*(heuristic[i]/total_heuristic) + cumul
                cumul += 1000*(heuristic[i]/total_heuristic)
            
            s = random.randint(0, 1000)
            ret = 0
            for i in range(self.vm_len):
                if (s > limits[i]):
                    ret = i+1
            return ret
    
    def preIterSetup(self):
        self.M = np.zeros((self.m, self.task_len, self.vm_len)) #Mapping from tasks to VMs: [n_ants, n_tasks, n_vms]
        self.vm_completetime = np.zeros((self.m, self.vm_len)) # [n_ants, n_vms] Sum of execution times for all tasks assigned to it
        self.c_time_i =np.zeros((self.m)) # Completion times for the set of tasks [n_ants]
        self.total_cost_i = np.zeros((self.m)) # Cost of using VMs [n_ants]  
        self.time_cf = np.zeros((self.m)) # Time Constraint Function: [n_ants]
        self.c_cost_i = np.zeros((self.m)) # Cost Constraint Function: [n_ants]
        self.L = np.zeros((self.m)) # w1*time_cf + w2*c_cost_i: [n_ants]
        
        self.current_total_exectime = np.zeros((self.m)) # Current total of all VM executon times
    
    def updatePheromone(self, a):
        for i in range(self.task_len):
            for j in range(self.vm_len):
                increment = 0.0
                if (self.M[a][i][j] == 1):
                    increment = 1.0 / float(self.L[a])
                self.Pher[i][j] = (1 - self.rho)*self.Pher[i][j] + self.rho*increment
    
    def updateGlobalPheromone(self):
        for i in range(self.task_len):
            for j in range(self.vm_len):
                increment = 0.0
                if (self.M_global_opt[i][j] == 1):
                    increment = 1.0 / float(self.L_global_opt)
                self.Pher[i][j] = (1 - self.rho1)*self.Pher[i][j] + self.rho1*increment
    
    def oneIteration(self):
        self.preIterSetup()
        for a in range(self.m):
            for t in range(self.task_len):
                chosen_vm = self.chooseVM(t,a)
                self.M[a][t][chosen_vm]  = 1
                self.vm_completetime[a][chosen_vm] +=  self.TRT[t][chosen_vm]
                self.current_total_exectime[a] += self.TRT[t][chosen_vm]
                if (self.vm_completetime[a][chosen_vm] > self.c_time_i[a]):
                    self.c_time_i[a] = self.vm_completetime[a][chosen_vm]
                self.total_cost_i[a] += self.TRT[t][chosen_vm] * self.VMs[chosen_vm]["cost"]
            self.time_cf[a] = (self.c_time_i[a] - self.c_time_min) / (self.c_time_max - self.c_time_min)
            self.c_cost_i[a] = ( self.total_cost_i[a] - self.total_cost_min ) / ( self.total_cost_max - self.total_cost_min)
            self.L[a] =  ((self.w1 * self.time_cf[a]) + (self.w2 * self.c_cost_i[a]))
            # Setting Data after all assignments of Tasks to VMs
        
        print ("Local Optimum is: ", min(self.L))
        for a in range(self.m):
            self.updatePheromone(a)
            if (self.L_global_opt == -1):
                self.L_global_opt = self.L[a]
                self.M_global_opt = self.M[a]
            elif (self.L[a] < self.L_global_opt):
                print ("New Optimum: ", self.L_global_opt, self.L[a])
                self.L_global_opt = self.L[a]
                self.M_global_opt = self.M[a]
        
        self.updateGlobalPheromone()
            
        
    def run(self):
        self.setInitialPheromone()
        for it in range(self.Nmax):
            self.oneIteration()


In [128]:
aco = ACO()
aco.loadData("vm_data.txt", "task_data.txt")

In [129]:
aco.run()

Local Optimum is:  1.75749049909
New Optimum:  1.80032649141 1.76507713485
New Optimum:  1.76507713485 1.7647855061
New Optimum:  1.7647855061 1.75749049909
Local Optimum is:  1.75638009946
New Optimum:  1.75749049909 1.75638009946
Local Optimum is:  1.75536166306
New Optimum:  1.75638009946 1.75536166306
Local Optimum is:  1.74575289372
New Optimum:  1.75536166306 1.74575289372
Local Optimum is:  1.74523481536
New Optimum:  1.74575289372 1.74523481536
Local Optimum is:  1.74523481536
Local Optimum is:  1.74342926417
New Optimum:  1.74523481536 1.74342926417
Local Optimum is:  1.74523481536
Local Optimum is:  1.74575289372
Local Optimum is:  1.74523481536
Local Optimum is:  1.74523481536
Local Optimum is:  1.74523481536
Local Optimum is:  1.74523481536
Local Optimum is:  1.74523481536
Local Optimum is:  1.74221422438
New Optimum:  1.74342926417 1.74221422438
Local Optimum is:  1.74221422438
Local Optimum is:  1.74273230274
Local Optimum is:  1.74221422438
Local Optimum is:  1.742732302

In [116]:
print (aco.L_global_opt)

1.74775624341
